In [ ]:
!pip install -U transformers accelerate datasets diffusers Pillow==9.4.0
from local_secrets import hf_token, wandb_key
from huggingface_hub import login
login(token=hf_token)

In [2]:
from utils import latent_to_PIL, dcae_scalingf, make_grid, batch_PIL_to_latent

In [3]:
from datasets import load_dataset, Dataset, DatasetDict

ds = load_dataset("evanarlian/imagenet_1k_resized_256", streaming=True)

README.md:   0%|          | 0.00/42.8k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

In [4]:
import torch
from diffusers import AutoencoderDC

dtype = torch.bfloat16
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"

model = "Efficient-Large-Model/Sana_600M_1024px_diffusers"
dcae = AutoencoderDC.from_pretrained(model, subfolder="vae", torch_dtype=dtype).to(device)

vae/config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm

hf_dataset = "g-ronimo/Imagenet-256-latents_dc-ae-f32c32-sana-1.0"
# hf_dataset = "g-ronimo/test2"
bs = 128
# upload_every = 20
splits = ["train", "test"]

for split in splits:
    num_batches=ds[split].info.splits[split].num_examples//bs
    upload_every = num_batches // 10
    ds_latents = []
    ds_part = 0
    print(f"split {split}, total batches {num_batches}, uploading every {upload_every}")
    
    for b_no, batch in tqdm(enumerate(ds[split].batch(batch_size=bs)), total=num_batches):
        images, labels = batch["image"], batch["label"]
        latents = batch_PIL_to_latent(images, dcae).float().cpu()
        for idx in range(len(labels)):
            label, latent = labels[idx], latents[idx,]
            ds_latents.append({"label": label, "latent": latent.numpy()})
        if (b_no+1) % upload_every == 0:
            print(f"Uploading part {ds_part} of split {split} to {hf_dataset}")
            Dataset.from_list(ds_latents).push_to_hub(hf_dataset, split=f"{split}.{ds_part}")
            ds_part+=1
            ds_latents=[]
    print(f"Uploading part {ds_part} of split {split} to {hf_dataset}")
    Dataset.from_list(ds_latents).push_to_hub(hf_dataset, split=f"{split}.{ds_part}")



In [ ]:
!runpodctl remove pod $RUNPOD_POD_ID

# Update: found a mistake
Used the wrong eval split, the one I took `test` does not contains labels, I need `val`

In [6]:
from tqdm import tqdm

hf_dataset = "g-ronimo/Imagenet-256-latents_dc-ae-f32c32-sana-1.0"
bs = 128
splits = ["val"]

for split in splits:
    num_batches=ds[split].info.splits[split].num_examples//bs
    ds_latents = []
    ds_part = 0
    print(f"split {split}, total batches {num_batches}")
    
    for b_no, batch in tqdm(enumerate(ds[split].batch(batch_size=bs)), total=num_batches):
        images, labels = batch["image"], batch["label"]
        latents = batch_PIL_to_latent(images, dcae).float().cpu()
        for idx in range(len(labels)):
            label, latent = labels[idx], latents[idx,]
            ds_latents.append({"label": label, "latent": latent.numpy()})

split val, total batches 390


391it [05:56,  1.10it/s]                         


In [7]:
print(f"Uploading split {split} to {hf_dataset}")
Dataset.from_list(ds_latents).push_to_hub(hf_dataset, split=f"{split}")

Uploading split val to g-ronimo/Imagenet-256-latents_dc-ae-f32c32-sana-1.0


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/50 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/464 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/g-ronimo/Imagenet-256-latents_dc-ae-f32c32-sana-1.0/commit/a2344b1a443b632c0a33ec8b600224767afdf813', commit_message='Upload dataset', commit_description='', oid='a2344b1a443b632c0a33ec8b600224767afdf813', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/g-ronimo/Imagenet-256-latents_dc-ae-f32c32-sana-1.0', endpoint='https://huggingface.co', repo_type='dataset', repo_id='g-ronimo/Imagenet-256-latents_dc-ae-f32c32-sana-1.0'), pr_revision=None, pr_num=None)